# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Karen Rodriguez Pinto
- Enlin Wei
- Yumi Minami 
- Kanami Hannah Tanaka

# Group Members IDs

- A15161856
- A12938737
- A12669124
- A########

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
# imports
%matplotlib inline

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# loads Kickstarters data into dataframe
df_kickstarters = pd.read_csv("Kickstarter_projects_Feb19.csv")

# Data Cleaning

We first dropped the column 'id' since it gives no useful information, just an ID associated with the project. We then dropped columns 'start_month' and 'end_month' since the information about the start and end months of the projects are already given in the dates in the 'launched_at' and 'deadline' columns and so is redundant.

In [3]:
df_kickstarters.drop(columns=['id', 'start_month', 'end_month'], inplace=True)

We then noticed that the **launched_at** and **deadline** columns contained strings which represented dates and times. It would be easier to work with datetime objects, so we created two new columns to replace the old ones with datetime objects instead of string representations of dates and times. We first created the new columns, then deleted the old ones and renamed the new columns to the names of the old ones because replacing in place took too long for the code to run.

In [4]:
# method to convert datetime strings in 'df_kickstarters' into datetime objects
def to_datetime(datetime_str):
    return dt.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')

In [5]:
# converts all datetime strings in df to datetime objects and adds them to new lists
# this step may take a while since there are almost 200,000 entries
launched_at_dt_format = []
deadline_dt_format = []
for i in range(0,len(df_kickstarters)):
    launched_at_dt_format.append(to_datetime(df_kickstarters.iloc[i]['launched_at']))
    deadline_dt_format.append(to_datetime(df_kickstarters.iloc[i]['deadline']))

In [6]:
# adds these lists to the dataframe
df_kickstarters['launched_at_dt_format'] = launched_at_dt_format
df_kickstarters['deadline_dt_format'] = deadline_dt_format

In [7]:
# drops the original 'launched_at' and 'deadline' columns since they are now redundant
df_kickstarters.drop(columns=['launched_at', 'deadline'], inplace=True)

In [8]:
# renames the new columns to the old names
df_kickstarters.rename(columns={'launched_at_dt_format':'launched_at', 'deadline_dt_format':'deadline'}, inplace=True)

Finally, we noticed some of the monetary amounts have more than 2 decimal places, which should be cleaned up since we don't work with monetary amounts more specific than 2 decimal places. So we decided to round the amounts to 2 decimal places.

In [9]:
df_kickstarters['goal_usd'] = df_kickstarters['goal_usd'].round(2)
df_kickstarters['usd_pledged'] = df_kickstarters['usd_pledged'].round(2)

# Data Analysis & Results

Since our project aims to analyze which factor is most important for the success of kickstarters, we will divide the dataframe into two separate ones, one with all the successful projects and one with all the failed projects.

In [10]:
df_successful = df_kickstarters[df_kickstarters['status']=='successful']
df_failed = df_kickstarters[df_kickstarters['status']=='failed']

In [11]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*